# IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

print("Setup complete")

Setup complete


# ARTIST RECOMMENDATION ENGINE

## Reading .csv file into variable 'artist'

In [2]:
Artist = pd.read_csv('data_by_artist.csv')
print(Artist.columns)
Artist.head()

Index(['artists', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'popularity', 'key', 'mode', 'count'],
      dtype='object')


,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,"""Cats"" 1981 Original London Cast",0.575083,0.442750,247260.000000,0.386336,0.022717,0.287708,-14.205417,0.180675,115.983500,0.334433,38.000000,5,1,12
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,33.076923,5,1,26
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.285714,0,1,7
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.444444,0,1,27
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.605444,0.437333,232428.111111,0.429333,0.037534,0.216111,-11.447222,0.086000,120.329667,0.458667,42.555556,11,1,9


## Dropping unnecessary columns

In [3]:
Artist.drop(['duration_ms','key','mode','count'],axis=1,inplace=True)

## Normalizing columns

In [4]:
Artist['popularity'] = Artist['popularity']/100
Artist['tempo'] = (Artist['tempo'] - 50)/100
Artist['loudness'] = (Artist['loudness'] + 60)/60

## User profile and recommendation matrix

In [5]:
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 
            'loudness', 'speechiness', 'tempo', 'valence', 'popularity']
def createUserPrefMatrix(artistRatingDict):
    
    artists = artistRatingDict.keys()
    artMat = Artist[Artist['artists'].isin(artists)]
    
    for artist, rating in artistRatingDict.items():
        artMat.loc[artMat['artists']==
                   artist,features] = artMat.loc[artMat['artists']==
                                                 artist,features].mul(rating,axis=0)
    
    userProfile = artMat.loc[:,features].sum(axis=0)
    normalized_userProfile = (userProfile/userProfile.sum())*10
    
    return normalized_userProfile

def createRecomMatrix(userProfile,artists):
    
    artMat = Artist[~Artist['artists'].isin(artists)]
    artMat.set_index('artists',inplace=True) 
    #print(userProfile)
    #print(artMat.head())
    
    recomMat = pd.DataFrame(artMat.values*userProfile.values, 
                            columns=artMat.columns, index=artMat.index)
    recomMat = recomMat.sum(axis=1)
    recomMat.sort_values(ascending = False,inplace=True)
    
    return recomMat

def recommend(artistRatingDict):
    
    userProfile = createUserPrefMatrix(artistRatingDict)
    
    recommendationMat = createRecomMatrix(userProfile,
                                          artistRatingDict.keys()) 
    
    return recommendationMat.head(10)


## Random user ratings

In [6]:
import random
artists = random.sample(list(Artist['artists']),k=10)
ratings = [10,10,8,5,9,2,3,7,6,10]
dictionary = dict(zip(artists, ratings))
print(dictionary)

{'Tony Mottola': 10, "Q'65": 10, 'Mickey Mouse': 8, 'Harold Land': 5, 'Avril Lavigne': 9, 'Marc Cohn': 2, 'Taeko Onuki': 3, 'Xristos Tselentis': 7, 'The Friends Of Distinction': 6, 'Raymond Scott': 10}


## Artist recommendation result

In [7]:
recommend(dictionary)

artists
Aymara pan-pipe orchestra          8.475883
Henry Blair                        8.451018
The Jiants                         8.443661
Hawkins C. All Star Jam Band       8.294508
DJ Mister Cee                      8.285331
Tim Armstrong                      8.284532
Kala Balch                         8.264396
Marco Marinangeli                  8.264396
Edmundo Ros                        8.213066
Banda Los Tamazulas De Culiacan    8.207989
dtype: float64